In [ ]:
!pip install SQLAlchemy
!apt install unixodbc-dev
!pip install pyodbc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  unixodbc-dev
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 217 kB of archives.
After this operation, 1,779 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 unixodbc-dev amd64 2.3.4-1.1ubuntu3 [217 kB]
Fetched 217 kB in 1s (396 kB/s)
Selecting previously unselected package unixodbc-dev:amd64.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../unixodbc-dev_2.3.4-1.1ubuntu3_amd64.deb ...
Unpacking unixodbc-dev:amd64 (2.3.4-1.1ubuntu3) ...
Setting up unixodbc-dev:amd64 (2.3.4-1.1ubuntu3) ...
     |████████████████████████████████| 276kB 5.7MB/s 
  Created wheel for pyodbc: filename=pyodbc-4.0.30-cp36-cp36m-linux_x86_64.whl size=272678 sha256=d79b14a06aa17f9d0433870da430ff3b8b38d3c71257e4e5e477bb10c4a431ef
  Stored in directo

In [ ]:
%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease [4,003 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:9 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd64 Packages [193 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key output should not be parsed (stdout is not a terminal)
100   983  100   983    0     0   7071      0 --:--:-- --:--:-- --:--:--  7021
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    79  100    79    0     0    929      0 --:--:-- --:--:-- --:--:--   940
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling

In [ ]:
import pyodbc 

pyodbc.drivers()

['ODBC Driver 17 for SQL Server']

In [ ]:
import json

# carregando arquivo com as chaves para o banco de dados
credentials = open('mssql_olist_data_keys.json')
credentials = json.load(credentials)

In [ ]:
try:
  conexion = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                            SERVER = credentials['mssql_server'],
                            DATABASE = credentials['mssql_database'],
                            UID = credentials['mssql_uid'],
                            PWD = credentials['mssql_pwd'])
  

  cursor = conexion.cursor()


  cursor.execute(
      """
      IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = N'customers_dataset')
      BEGIN
        CREATE TABLE customers_dataset (
          id_num INT NOT NULL IDENTITY (1, 1) PRIMARY KEY,
          customer_id VARCHAR(100),
          customer_unique_id VARCHAR(100),
          customer_zip_code_prefix int,
          customer_city varchar(100),
          customer_state varchar(100)
      )
      END;
      """
  )


  cursor.execute(
      """
      IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = N'geolocation_dataset')
      BEGIN
        CREATE TABLE geolocation_dataset (
          id_num INT NOT NULL IDENTITY (1, 1) PRIMARY KEY,
          geolocation_zip_code_prefix INT,
          geolocation_lat FLOAT,
          geolocation_lng FLOAT,
          geolocation_city VARCHAR(100),
          geolocation_state VARCHAR(100)
      )
      END;
      """
  )


  cursor.execute(
      """
      IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = N'order_items_dataset')
      BEGIN
        CREATE TABLE order_items_dataset (
          id_num INT NOT NULL IDENTITY (1, 1) PRIMARY KEY,
          order_id VARCHAR(100),
          order_item_id INT,
          product_id VARCHAR(100),
          seller_id VARCHAR(100),
          shipping_limit_date DATETIME,
          price FLOAT,
          freight_value FLOAT
      )
      END;
      """
  )


  cursor.execute(
      """
      IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = N'order_payments_dataset')
      BEGIN
        CREATE TABLE order_payments_dataset (
          id_num INT NOT NULL IDENTITY (1, 1) PRIMARY KEY,
          order_id VARCHAR(100),
          payment_sequential INT,
          payment_type VARCHAR(100),
          payment_installments INT,
          payment_value FLOAT
      )
      END;
      """
  )


  cursor.execute(
      """
      IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = N'order_reviews_dataset')
      BEGIN
        CREATE TABLE order_reviews_dataset (
          id_num INT NOT NULL IDENTITY (1, 1) PRIMARY KEY,
          review_id VARCHAR(100),
          order_id VARCHAR(100),
          review_score INT,
          review_comment_title VARCHAR(100),
          review_comment_message VARCHAR(100),
          review_creation_date DATETIME,
          review_answer_timestamp DATETIME        
      )
      END;
      """
  )


  cursor.execute(
      """
      IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = N'orders_dataset')
      BEGIN
        CREATE TABLE orders_dataset (
          id_num INT NOT NULL IDENTITY (1, 1) PRIMARY KEY,
          order_id VARCHAR(100),
          customer_id VARCHAR(100),
          order_status VARCHAR(100),
          order_purchase_timestamp DATETIME,
          order_approved_at DATETIME,
          order_delivered_at DATETIME,
          order_delivered_carrier_date DATETIME,
          order_delivered_customer_date DATETIME,
          order_estimated_delivery_date DATETIME
      )
      END;
      """
  )


  cursor.execute(
      """
      IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = N'products_dataset')
      BEGIN
        CREATE TABLE products_dataset (
          id_num INT NOT NULL IDENTITY (1, 1) PRIMARY KEY,
          product_id VARCHAR(100),
          product_category_name VARCHAR(100),
          product_name_lenght FLOAT,
          product_description_lenght FLOAT,
          products_photos_qty FLOAT,
          product_weight_g FLOAT,
          product_lenght_cm FLOAT,
          product_height_cm FLOAT,
          product_width_cm FLOAT
      )
      END;
      """
  )


  cursor.execute(
      """
      IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = N'sellers_dataset')
      BEGIN
        CREATE TABLE sellers_dataset (
          id_num INT NOT NULL IDENTITY (1, 1) PRIMARY KEY,
          seller_id VARCHAR(100),
          seller_zip_code_prefix INT,
          seller_city VARCHAR(100),
          seller_state VARCHAR(100)
      )
      END; 
      """
  )


  cursor.execute(
      """
      IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = N'product_category_name_translation')
      BEGIN
        CREATE TABLE product_category_name_translation (
          id_num INT NOT NULL IDENTITY (1, 1) PRIMARY KEY,
          product_category_name VARCHAR(100),
          product_category_name_english VARCHAR(100)
      )
      END; 
      """
  )

  conexion.commit()
  conexion.close()

except Exception as e:
    print("Ocorreu um erro na tentativa de conexão: ", e)

In [ ]:
try:
  conexion = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                          SERVER = credentials['mssql_server'],
                          DATABASE = credentials['mssql_database'],
                          UID = credentials['mssql_uid'],
                          PWD = credentials['mssql_pwd'])

  cursor = conexion.cursor()

  cursor.execute(
      """
      SELECT * FROM INFORMATION_SCHEMA.TABLES;
      """
      )

  tables = cursor.fetchall()
  conexion.close()

  for table in tables:
    print(table)


    
except Exception as e:
  print("Ocorreu um erro na tentativa de conexão: ", e)

('olist_data', 'dbo', 'geolocation_dataset', 'BASE TABLE')
('olist_data', 'dbo', 'order_items_dataset', 'BASE TABLE')
('olist_data', 'dbo', 'order_payments_dataset', 'BASE TABLE')
('olist_data', 'dbo', 'order_reviews_dataset', 'BASE TABLE')
('olist_data', 'dbo', 'orders_dataset', 'BASE TABLE')
('olist_data', 'dbo', 'products_dataset', 'BASE TABLE')
('olist_data', 'dbo', 'sellers_dataset', 'BASE TABLE')
('olist_data', 'dbo', 'product_category_name_translation', 'BASE TABLE')
('olist_data', 'dbo', 'customers_dataset', 'BASE TABLE')


In [ ]:
import pandas as pd

# olist_customers_dataset.csv
customers_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_customers_dataset.csv'
customers_dataset = pd.read_csv(customers_dataset_url)

# olist_geolocation_dataset.csv
geolocation_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_geolocation_dataset.csv'
geolocation_dataset = pd.read_csv(geolocation_dataset_url)

# olist_order_items_dataset.csv
order_items_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_order_items_dataset.csv'
order_items_dataset = pd.read_csv(order_items_dataset_url)

# olist_order_payments_dataset.csv
order_payments_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_order_payments_dataset.csv'
order_payments_dataset = pd.read_csv(order_payments_dataset_url)

# olist_order_reviews_dataset.csv
order_reviews_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_order_reviews_dataset.csv'
order_reviews_dataset = pd.read_csv(order_reviews_dataset_url)

# olist_orders_dataset.csv
orders_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_orders_dataset.csv'
orders_dataset = pd.read_csv(orders_dataset_url)

# olist_products_dataset.csv
products_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_products_dataset.csv'
products_dataset = pd.read_csv(products_dataset_url)

# olist_sellers_dataset.csv
sellers_dataset_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/olist_sellers_dataset.csv'
sellers_dataset = pd.read_csv(sellers_dataset_url)

# product_category_name_translation.csv
product_category_name_translation_url = 'https://raw.githubusercontent.com/olist/work-at-olist-data/master/datasets/product_category_name_translation.csv'
product_category_name_translation = pd.read_csv(product_category_name_translation_url)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
def mssql_olist_data_bulk_insert_dataframe(dataframe, table_name):

  import numpy as np
  import urllib
  import sqlalchemy
  from datetime import datetime
  import pandas as pd
  import pyodbc 

  connection_string = "DRIVER={};SERVER={};DATABASE={};UID={};PWD={}".format('ODBC Driver 17 for SQL Server',
                                                                             credentials['mssql_server'],
                                                                             credentials['mssql_database'],
                                                                             credentials['mssql_uid'],                                   
                                                                             credentials['mssql_pwd'])

  batch_size = 200000
  batches = np.ceil(dataframe.shape[0] / batch_size)

  for i in range(int(batches)):
    
    if i == batches - 1:
      inicio = i * batch_size
      final = dataframe.shape[0]
    
    else:
      inicio = i * batch_size
      final = inicio + batch_size

    temp_dataframe = dataframe.iloc[inicio: final]

    params = urllib.parse.quote_plus(connection_string)
    engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params, fast_executemany=True)

    temp_dataframe.to_sql(table_name, engine, if_exists='append', index=False)

    # print(f'Batch: {i+1} de {int(batches)}')


In [ ]:
def mssql_clear_table(table_name):

  try:
      conexion = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                                SERVER = credentials['mssql_server'],
                                DATABASE = credentials['mssql_database'],
                                UID = credentials['mssql_uid'],
                                PWD = credentials['mssql_pwd'])
      

      cursor = conexion.cursor()
      cursor.execute(f'delete from {table_name};')

      conexion.commit()
      conexion.close()

      
  except Exception as e:
      print("Ocorreu um erro na tentativa de conexão: ", e) 

In [ ]:
# customers_dataset
mssql_clear_table('customers_dataset')
mssql_olist_data_bulk_insert_dataframe(customers_dataset, 'customers_dataset')

# geolocation_dataset
mssql_clear_table('geolocation_dataset')
mssql_olist_data_bulk_insert_dataframe(geolocation_dataset, 'geolocation_dataset')

# order_items_dataset
mssql_clear_table('order_items_dataset')
mssql_olist_data_bulk_insert_dataframe(order_items_dataset, 'order_items_dataset')

# order_payments_dataset
mssql_clear_table('order_payments_dataset')
mssql_olist_data_bulk_insert_dataframe(order_payments_dataset, 'order_payments_dataset')

# order_reviews_dataset
mssql_clear_table('order_reviews_dataset')
mssql_olist_data_bulk_insert_dataframe(order_reviews_dataset, 'order_reviews_dataset')

# orders_dataset
mssql_clear_table('orders_dataset')
mssql_olist_data_bulk_insert_dataframe(orders_dataset, 'orders_dataset')

# products_dataset
mssql_clear_table('products_dataset')
mssql_olist_data_bulk_insert_dataframe(products_dataset, 'products_dataset')

# sellers_dataset
mssql_clear_table('sellers_dataset')
mssql_olist_data_bulk_insert_dataframe(sellers_dataset, 'sellers_dataset')

# product_category_name_translation
mssql_clear_table('product_category_name_translation')
mssql_olist_data_bulk_insert_dataframe(product_category_name_translation, 'product_category_name_translation')

In [ ]:
try:
  conexion = pyodbc.connect(DRIVER = '{ODBC Driver 17 for SQL Server}',
                            SERVER = credentials['mssql_server'],
                            DATABASE = credentials['mssql_database'],
                            UID = credentials['mssql_uid'],
                            PWD = credentials['mssql_pwd'])


  results = pd.read_sql('SELECT * FROM INFORMATION_SCHEMA.TABLES;', conexion)

  for tabela in results['TABLE_NAME']:
    # print(tabela)
    print(tabela, pd.read_sql(f'SELECT * FROM {tabela};', conexion).shape[0])

  conexion.close()
   
    
except Exception as e:
  print("Ocorreu um erro na tentativa de conexão: ", e)



geolocation_dataset 1000163
order_items_dataset 112650
order_payments_dataset 103886
order_reviews_dataset 100000
orders_dataset 99441
products_dataset 32951
sellers_dataset 3095
product_category_name_translation 71
customers_dataset 99441
